In [72]:
import numpy as np
import pandas as pd
import random

ROOT = '/home/robert/Projects/NCAA2021'

In [73]:
RegularSeason = pd.read_csv(f'{ROOT}/Data/MRegularSeasonDetailedResults.csv')
Tourney = pd.read_csv(f'{ROOT}/Data/MNCAATourneyCompactResults.csv')
TeamIDs = pd.read_csv(f'{ROOT}/Data/MTeams.csv')
Rankings = pd.read_csv(f'{ROOT}/Data/MMasseyOrdinals.csv')

In [74]:
Train = RegularSeason
Train.drop(['DayNum', 'WLoc', 'NumOT'], axis=1, inplace=True)

In [75]:
def randomize_team(WTeamID):
    if random.randint(0,1):
        return WTeamID
    else:
        return np.NaN
RegularSeason

,Season,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,1104,68,1328,62,27,58,3,14,11,...,10,16,22,10,22,8,18,9,2,20
1,2003,1272,70,1393,63,26,62,8,20,10,...,24,9,20,20,25,7,12,8,6,16
2,2003,1266,73,1437,61,24,58,8,18,17,...,26,14,23,31,22,9,12,2,5,23
3,2003,1296,56,1457,50,18,38,3,9,17,...,22,8,15,17,20,9,19,4,3,23
4,2003,1400,77,1208,71,30,61,6,14,11,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100418,2022,1400,79,1242,76,28,67,3,20,20,...,13,15,23,5,24,10,15,3,5,21
100419,2022,1411,66,1126,63,24,59,2,20,16,...,21,15,24,5,23,10,19,13,2,23
100420,2022,1422,68,1441,49,23,56,13,32,9,...,24,8,11,10,18,5,16,8,2,12
100421,2022,1438,69,1181,68,31,65,2,12,5,...,17,18,22,11,25,14,14,3,9,11


In [76]:
Train['Team1'] = Train['WTeamID'].apply(lambda x: randomize_team(x))
Train['Team1'].fillna(Train.LTeamID, inplace=True)
Train['Team1'] = Train['Team1'].astype(int)
Train['Team2'] = np.where(Train['WTeamID']==Train['Team1'], Train['LTeamID'], Train['WTeamID'])
Train['Team1 Pts'] = np.where(Train['Team1']==Train['WTeamID'], Train['WScore'], Train['LScore'])
Train['Team2 Pts'] = np.where(Train['Team2']==Train['WTeamID'], Train['WScore'], Train['LScore'])
Train

,Season,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,...,LDR,LAst,LTO,LStl,LBlk,LPF,Team1,Team2,Team1 Pts,Team2 Pts
0,2003,1104,68,1328,62,27,58,3,14,11,...,22,8,18,9,2,20,1328,1104,62,68
1,2003,1272,70,1393,63,26,62,8,20,10,...,25,7,12,8,6,16,1272,1393,70,63
2,2003,1266,73,1437,61,24,58,8,18,17,...,22,9,12,2,5,23,1437,1266,61,73
3,2003,1296,56,1457,50,18,38,3,9,17,...,20,9,19,4,3,23,1296,1457,56,50
4,2003,1400,77,1208,71,30,61,6,14,11,...,15,12,10,7,1,14,1400,1208,77,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100418,2022,1400,79,1242,76,28,67,3,20,20,...,24,10,15,3,5,21,1242,1400,76,79
100419,2022,1411,66,1126,63,24,59,2,20,16,...,23,10,19,13,2,23,1126,1411,63,66
100420,2022,1422,68,1441,49,23,56,13,32,9,...,18,5,16,8,2,12,1441,1422,49,68
100421,2022,1438,69,1181,68,31,65,2,12,5,...,25,14,14,3,9,11,1438,1181,69,68


In [77]:
def get_team_rank(team_id, year):
    temp = Rankings[Rankings.TeamID==team_id]
    temp = temp[temp.Season==year]
    return temp['OrdinalRank'].sum() / len(temp.index)
Train['Team1 Rank'] = Train.apply(lambda x: get_team_rank(x['Team1'], x['Season']), axis=1)
Train['Team2 Rank'] = Train.apply(lambda x: get_team_rank(x['Team2'], x['Season']), axis=1)

In [78]:
def split_season(team, year):
    win_df = RegularSeason[RegularSeason.WTeamID==team]
    win_df = win_df[win_df.Season==year]
    loss_df = RegularSeason[RegularSeason.LTeamID==team]
    loss_df = loss_df[loss_df.Season==year]
    num_games = len(win_df) + len(loss_df)
    if num_games == 0:
        num_games = 1
    return win_df, loss_df, num_games

In [79]:
def get_win_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return len(win_df) / num_games
Train['Team1 Win%'] = Train.apply(lambda x:get_win_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 Win%'] = Train.apply(lambda x:get_win_perc(x['Team2'], x['Season']), axis=1)

In [80]:
def get_points_avg(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WScore'].sum() + loss_df['LScore'].sum())/num_games
Train['Team1 Avg Pts'] = Train.apply(lambda x:get_points_avg(x['Team1'], x['Season']), axis=1)
Train['Team2 Avg Pts'] = Train.apply(lambda x:get_points_avg(x['Team2'], x['Season']), axis=1)

In [81]:
def get_points_a_avg(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LScore'].sum() + loss_df['WScore'].sum())/num_games
Train['Team1 Avg Pts A'] = Train.apply(lambda x:get_points_a_avg(x['Team1'], x['Season']), axis=1)
Train['Team2 Avg Pts A'] = Train.apply(lambda x:get_points_a_avg(x['Team2'], x['Season']), axis=1)

In [82]:
def get_fg_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WFGM'].sum() + loss_df['LFGM'].sum()) / (win_df['WFGA'].sum()+loss_df['LFGA'].sum())
Train['Team1 Fg%'] = Train.apply(lambda x:get_fg_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 Fg%'] = Train.apply(lambda x:get_fg_perc(x['Team2'], x['Season']), axis=1)

In [83]:
def get_fg_perc_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LFGM'].sum() + loss_df['WFGM'].sum()) / (win_df['LFGA'].sum()+loss_df['WFGA'].sum())
Train['Team1 Fg% A'] = Train.apply(lambda x:get_fg_perc_a(x['Team1'], x['Season']), axis=1)
Train['Team2 Fg% A'] = Train.apply(lambda x:get_fg_perc_a(x['Team2'], x['Season']), axis=1)

In [84]:
def get_three_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WFGM3'].sum() + loss_df['LFGM3'].sum()) / (win_df['WFGA3'].sum()+loss_df['LFGA3'].sum())
Train['Team1 3%'] = Train.apply(lambda x:get_three_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 3%'] = Train.apply(lambda x:get_three_perc(x['Team2'], x['Season']), axis=1)

In [85]:
def get_three_perc_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LFGM3'].sum() + loss_df['WFGM3'].sum()) / (win_df['LFGA3'].sum()+loss_df['WFGA3'].sum())
Train['Team1 3% A'] = Train.apply(lambda x:get_three_perc_a(x['Team1'], x['Season']), axis=1)
Train['Team2 3% A'] = Train.apply(lambda x:get_three_perc_a(x['Team2'], x['Season']), axis=1)

In [86]:
def get_freethrows(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return win_df['WFTM'].sum()+loss_df['LFTM'].sum()
Train['Team1 FT'] = Train.apply(lambda x:get_freethrows(x['Team1'], x['Season']), axis=1)
Train['Team2 FT'] = Train.apply(lambda x:get_freethrows(x['Team2'], x['Season']), axis=1)

In [87]:
def get_freethrows_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return win_df['LFTM'].sum()+loss_df['WFTM'].sum()
Train['Team1 FT A'] = Train.apply(lambda x:get_freethrows_a(x['Team1'], x['Season']), axis=1)
Train['Team2 FT A'] = Train.apply(lambda x:get_freethrows_a(x['Team2'], x['Season']), axis=1)

In [88]:
def get_or(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WOR'].sum() + loss_df['LOR'].sum()) / num_games
Train['Team1 OR'] = Train.apply(lambda x:get_or(x['Team1'], x['Season']), axis=1)
Train['Team2 OR'] = Train.apply(lambda x:get_or(x['Team2'], x['Season']), axis=1)

In [89]:
def get_or_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LOR'].sum() + loss_df['WOR'].sum()) / num_games
Train['Team1 OR A'] = Train.apply(lambda x:get_or_a(x['Team1'], x['Season']), axis=1)
Train['Team2 OR A'] = Train.apply(lambda x:get_or_a(x['Team2'], x['Season']), axis=1)

In [90]:
def get_dr(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WDR'].sum() + loss_df['LDR'].sum()) / num_games
Train['Team1 DR'] = Train.apply(lambda x:get_dr(x['Team1'], x['Season']), axis=1)
Train['Team2 DR'] = Train.apply(lambda x:get_dr(x['Team2'], x['Season']), axis=1)

In [91]:
def get_dr_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LDR'].sum() + loss_df['WDR'].sum()) / num_games
Train['Team1 DR A'] = Train.apply(lambda x:get_dr_a(x['Team1'], x['Season']), axis=1)
Train['Team2 DR A'] = Train.apply(lambda x:get_dr_a(x['Team2'], x['Season']), axis=1)

In [92]:
def get_to(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WTO'].sum() + loss_df['LTO'].sum()) / num_games
Train['Team1 TO'] = Train.apply(lambda x:get_to(x['Team1'], x['Season']), axis=1)
Train['Team2 TO'] = Train.apply(lambda x:get_to(x['Team2'], x['Season']), axis=1)

In [93]:
def get_ta(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LTO'].sum() + loss_df['WTO'].sum()) / num_games
Train['Team1 TA'] = Train.apply(lambda x:get_ta(x['Team1'], x['Season']), axis=1)
Train['Team2 TA'] = Train.apply(lambda x:get_ta(x['Team2'], x['Season']), axis=1)

In [94]:
Train = Train[[col for col in Train.columns if col != 'Team1 Pts'] + ['Team1 Pts']]
Train = Train[[col for col in Train.columns if col != 'Team2 Pts'] + ['Team2 Pts']]
Train.dropna(inplace=True)
Train.to_csv(f'{ROOT}/Data/Train.csv')

In [95]:
Train

,Season,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,...,Team1 DR,Team2 DR,Team1 DR A,Team2 DR A,Team1 TO,Team2 TO,Team1 TA,Team2 TA,Team1 Pts,Team2 Pts
0,2003,1104,68,1328,62,27,58,3,14,11,...,24.966667,23.928571,22.400000,22.642857,11.800000,13.285714,13.700000,13.857143,62,68
1,2003,1272,70,1393,63,26,62,8,20,10,...,25.965517,26.896552,23.586207,22.275862,13.793103,13.620690,15.068966,14.448276,70,63
2,2003,1266,73,1437,61,24,58,8,18,17,...,23.700000,24.071429,22.900000,18.571429,16.033333,13.571429,16.966667,12.500000,61,73
3,2003,1296,56,1457,50,18,38,3,9,17,...,22.645161,23.964286,20.096774,23.821429,17.000000,14.642857,14.451613,15.642857,56,50
4,2003,1400,77,1208,71,30,61,6,14,11,...,26.142857,24.592593,22.000000,23.740741,13.428571,11.555556,14.178571,13.444444,77,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100418,2022,1400,79,1242,76,28,67,3,20,20,...,24.695652,22.416667,20.173913,19.750000,12.695652,10.958333,12.739130,14.791667,76,79
100419,2022,1411,66,1126,63,24,59,2,20,16,...,21.454545,24.900000,24.863636,21.750000,12.772727,14.300000,12.909091,11.650000,63,66
100420,2022,1422,68,1441,49,23,56,13,32,9,...,24.708333,23.434783,26.333333,20.478261,12.291667,12.913043,9.916667,11.608696,49,68
100421,2022,1438,69,1181,68,31,65,2,12,5,...,21.083333,26.782609,20.875000,21.739130,9.333333,10.434783,10.375000,11.695652,69,68
